In [21]:
#matplotlib inline
import random
import torch
import matplotlib.pyplot as plt
from setuptools.command.saveopts import saveopts
from torch import Tensor

In [5]:



def synthetic_data(w, b, num_examples):
    """
    Generate y = Xw + b + noise \n
    生成线性回归的合成数据集 \n
    # w: 权重向量，形状为 (n,1) \n
    # b: 偏置项，标量 \n
    # num_examples: 样本数量
    """

    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)


    # 返回特征和标签
    # X: 特征矩阵，形状为 (num_examples, len(w))
    # y: 标签向量，形状为 (num_examples,)
    # 将y转换为列向量
    return X, y.reshape((-1, 1))

def data_iter(batch_size, _features, _labels):
    """
    生成数据迭代器，每次从数据样本中抽取少量样本用于更新模型参数 \n
    使用yield关键字返回一个生成器对象 \n
    yield 关键字的作用是将函数变成生成器（generator），每次调用生成器的 __next__() 方法（如在 for 循环中），就会从上一次 yield 处继续执行，返回一个批次的数据，而不是一次性返回所有数据。这样可以节省内存，并且方便处理大规模数据集。\n
    # batch_size: 批量大小
    # features: 特征矩阵
    # labels: 标签向量
    """
    num_examples = len(_features)
    index = list(range(num_examples))
    random.shuffle(index)

    # 将特征和标签按照随机顺序重新排列
    # 根据batch_size将数据分成小批量
    for i in range(0, num_examples, batch_size):
        # 计算当前批次的索引
        # min(i + batch_size, num_examples) 确保不会超出范围
        batch = torch.tensor(index[i:min(i + batch_size, num_examples)])
        # index_select(0, batch_index) 从第0维（行）选择指定的索引
        # 返回一个新的张量，其中包含指定索引的行
        # yield 返回一个生成器对象，每次迭代返回一个批次的特征和标签
        # 下次调用时会从上次的yield处继续执行
        yield _features.index_select(0, batch), _labels.index_select(0, batch)



In [25]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)


In [26]:
batch_size = 10

for X,y in data_iter(batch_size, features, labels):
    print(X, '\n', y)
    break  # 只打印第一个批次的数据

tensor([[-1.1194,  1.3199],
        [-0.2612, -1.1244],
        [-0.5767,  0.3896],
        [-0.4884, -1.0144],
        [-1.8603,  0.5096],
        [ 0.0821,  1.3456],
        [-0.3518,  0.7616],
        [-0.1881,  0.3565],
        [-0.6909, -0.7781],
        [ 1.6174,  0.9306]]) 
 tensor([[-2.5369],
        [ 7.4994],
        [ 1.7232],
        [ 6.6892],
        [-1.2628],
        [-0.2121],
        [ 0.9193],
        [ 2.6291],
        [ 5.4752],
        [ 4.2654]])


定义模型、损失函数、优化器

In [45]:
def linear_reg(X, w, b):
    """
    线性回归模型 \n
    # X: 特征矩阵，形状为 (num_examples, n) \n
    # w: 权重向量，形状为 (n,1) \n
    # b: 偏置项，标量 \n
    """
    return torch.matmul(X, w) + b

def squared_loss(y_hat, y):
    """
    均方误差损失函数 \n
    # y_hat: 模型预测值，形状为 (num_examples, 1) \n
    # y: 真实标签，形状为 (num_examples, 1) \n
    """
    return (y_hat - y.view(y_hat.shape)) ** 2 / 2

def sgd(params, learn_rate, batch_size):
    """
    随机梯度下降优化器 \n
    公式: \n
    w = w - learn_rate * (1/batch_size) * ∇L(w) \n

    # params: 模型参数列表 \n
    # learn_rate: 学习率 \n
    # batch_size: 批量大小 \n
    """
    with torch.no_grad(): # 禁用梯度计算
        # 遍历每个参数，更新其值
        for param in params:
            # 使用梯度下降更新参数
            # param.grad: 当前参数的梯度
            # learn_rate: 学习率
            # batch_size: 批量大小
            param -= learn_rate * param.grad / batch_size
            param.grad.zero_()  # 清除梯度

训练过程中，将进行如下迭代
- 初始化参数
- 重复以下步骤
    - 计算梯度
    - 更新参数

In [48]:
w = torch.normal(0, 0.01, size=(2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
lr = 0.03
num_epochs = 10
net = linear_reg
loss = squared_loss

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        # 前向传播
        y_hat = net(X, w, b)
        # 计算损失
        l = loss(y_hat, y)
        # 反向传播
        l.sum().backward()
        # 更新参数
        sgd([w, b], lr, batch_size)

    # 打印每个epoch的损失
    train_l = loss(net(features, w, b), labels)
    print(f'epoch {epoch + 1}, loss {train_l.mean():f}')

print(f'w的估计误差为：{true_w - w.reshape(true_w.shape)}')
print(f'b的估计误差为：{true_b - b}')

epoch 1, loss 0.037658
epoch 2, loss 0.000140
epoch 3, loss 0.000052
epoch 4, loss 0.000052
epoch 5, loss 0.000052
epoch 6, loss 0.000052
epoch 7, loss 0.000052
epoch 8, loss 0.000052
epoch 9, loss 0.000052
epoch 10, loss 0.000052
w的估计误差为：tensor([0.0004, 0.0005], grad_fn=<SubBackward0>)
b的估计误差为：tensor([-0.0002], grad_fn=<RsubBackward1>)


重启内核

线性回归的简洁实现

In [11]:
import numpy as np
import torch
from torch.utils import data


In [12]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

In [13]:
def load_array(data_arrays, batch_size, is_train=True):
    """
    将数据加载到DataLoader中 \n
    通过DataLoader可以方便地迭代数据 \n
    在每个迭代中，DataLoader会返回一个批次的数据 \n
    # data_arrays: 数据数组，包含特征和标签 \n
    # batch_size: 批量大小 \n
    # is_train: 是否为训练数据 \n
    """
    dataset = data.TensorDataset(*data_arrays)  # 创建TensorDataset
    return data.DataLoader(dataset, batch_size, shuffle=is_train)  # 返回DataLoader

batch_size = 10
# 使用load_array函数加载数据
# 获得一个DataLoader对象，可以用于迭代数据
data_iter = load_array((features, labels), batch_size)

In [14]:
from torch import  nn
# 定义线性回归模型
# 使用Sequential容器来构建模型 \n
# nn.Linear(2, 1) 表示输入特征维度为2，输出特征维度为1 \n

net = nn.Sequential(nn.Linear(2,1))

net[0].weight.data.normal_(0, 0.01)  # 初始化权重
net[0].bias.data.fill_(0)  # 初始化偏置

loss = nn.MSELoss()  # 定义损失函数为均方误差损失

trainer = torch.optim.SGD(net.parameters(), lr=0.03)  # 定义优化器为随机梯度下降


训练模型

In [15]:
num_epochs = 3

for epoch in range(num_epochs):
    for X, y in data_iter:
        # 前向传播
        y_hat = net(X)
        # 计算损失
        l = loss(y_hat, y)
        # 反向传播
        trainer.zero_grad()  # 清除梯度
        l.backward()  # 计算梯度
        trainer.step()  # 更新参数

    # 打印每个epoch的损失
    train_l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {train_l.mean():f}')

epoch 1, loss 0.000327
epoch 2, loss 0.000097
epoch 3, loss 0.000097


In [17]:
w = net[0].weight.data
b = net[0].bias.data
print(f'w的估计误差为：{true_w - w.reshape(true_w.shape)}')
print(f'b的估计误差为：{true_b - b}')

w的估计误差为：tensor([-0.0005,  0.0001])
b的估计误差为：tensor([0.0001])


IndexError: index 1 is out of range